## Install a pip package in the current Jupyter kernel

In [1]:
import sys
!{sys.executable} -m pip install pokebase

The directory '/home/jovyan/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/jovyan/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
  Running setup.py install for pokebase ... error
    Complete output from command /opt/conda/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-dhccjgmn/pokebase/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-d7qx53mb/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_py
    creating bui

## Import PokeAPI wrapper library for  Python 3

In [2]:
import pokebase as pb

ModuleNotFoundError: No module named 'pokebase'

## Quick lookup

In [ ]:
rayquaza = pb.pokemon('rayquaza')
print(rayquaza.height)
print(rayquaza.weight)


In [ ]:
rayquaza.types

In [3]:
rayquaza.stats

NameError: name 'rayquaza' is not defined

## Fetch txt file from Smogon

In [ ]:
from urllib.request import urlopen
import json

data = urlopen("https://www.smogon.com/stats/2019-03/gen7ubers-0.txt")
usage = []
for line in data:
  decoded = line.decode('utf-8')
  decoded = decoded.replace(" ", "")
  if decoded.startswith('|'):
    usage.append(decoded.split('|')[1:-1])

In [ ]:
usage

## Use pandas to visualize Smogon competitive usage data

In [ ]:
import pandas as pd

In [ ]:
smogon_df = pd.DataFrame(usage[1:], columns=usage[0])
smogon_df

# usage - weighted based on matchmaking rating
# raw - unweighted usage (on team)
# real - actually used in battle

In [ ]:
statsDic = {}
for pokemon in smogon_df['Pokemon']:
  pokemon_stats = []
  pokemon_info = pb.pokemon(pokemon.lower())
  for i in (pokemon_info.stats):
    pokemon_stats.append(str(i)[14:17])
  statsDic[pokemon] = pokemon_stats


## Obtain list of legendary and mystical pokemon from Bulbapedia

In [ ]:
import requests
from bs4 import BeautifulSoup

# scrape list of legendary and mystical pokemon from bulbapedia
page = requests.get("https://bulbapedia.bulbagarden.net/wiki/Legendary_Pok%C3%A9mon")
soup = BeautifulSoup(page.content, 'html.parser')

# list of <a> tags of legendary pokemon
a_list = soup.select('td a[title*=(Pokémon)]')

# extract text from <a> tags
legend_list = [lp.get_text() for lp in a_list]

# data cleaning: replace whitespace with hyphen, remove colon
legend_list = [p.lower().replace(' ', '-').replace(':', '') for p in legend_list]
legend_list

## Get itemized list for pokemon stats

In [ ]:
import re
pokemonList = []
pokemonList.append(['ID', 'Pokemon', 'Legendary', 'Stat Total', 'ATK Sum', 'DEF Sum', 'Height', 'Weight'])

for pokemonID in range(1,100):
  pokemon_stats = []
  pokemon_info = pb.pokemon(pokemonID)
  
  pokemon_stats.append(pokemonID) # Add ID
  pokemon_stats.append(pokemon_info.name) # Add Name
  pokemon_stats.append(pokemon_info.name in legend_list) # Add Legendary or Not
  pokemon_stats.extend([0,1,2]) # Add categories for stat totals
  pokemon_stats.append(pokemon_info.height) # Add Height
  pokemon_stats.append(pokemon_info.weight) # Add Weight

  j = 0 # Counter because I didnt want to change my code anymore
  for i in (pokemon_info.stats):
    stringStats = str(i)
    value = [int(s) for s in re.findall(r'\b\d+\b', stringStats)[0:20]]
    pokemon_stats[2] += value[0]
    pokemon_stats[3+(j%2)] += value[0]
    j += 1
    
  pokemonList.append(pokemon_stats)

statsDF = pd.DataFrame(pokemonList[1:], columns=pokemonList[0])
statsDF
 

## After scraping and cleaning, test for validity by searching for each name in the API

In [ ]:
for p in legend_list:
  try:
    pb.pokemon(p)
  except ValueError:
    print("Cannot find", p)

## The above Pokemon have alternate forms and we must handle each manually

P.S. Meltan and Melmetal haven't been added to the database at the moment. https://github.com/PokeAPI/pokeapi/issues/414

In [ ]:
# deoxys: deoxys-normal, deoxys-attack, deoxys-defense, deoxys-speed
try:
  legend_list.remove('deoxys')
  legend_list.extend(['deoxys-normal', 'deoxys-attack', 'deoxys-defense', 'deoxys-speed'])
except ValueError:
  print('Error')

# giratina: giratina-altered, giratina-origin
try:
  legend_list.remove('giratina')
  legend_list.extend(['giratina-altered', 'giratina-origin'])
except ValueError:
  print('Error')

# shaymin: shaymin-land, shaymin-sky
try:
  legend_list.remove('shaymin')
  legend_list.extend(['shaymin-land', 'shaymin-sky'])
except ValueError:
  print('Error')

# tornadus: tornadus-incarnate, tornadus-therian
try:
  legend_list.remove('tornadus')
  legend_list.extend(['tornadus-incarnate', 'tornadus-therian'])
except ValueError:
  print('Error')

# thundurus: thundurus-incarnate, thundurus-therian
try:
  legend_list.remove('thundurus')
  legend_list.extend(['thundurus-incarnate', 'thundurus-therian'])
except ValueError:
  print('Error')

# landorus: landorus-incarnate, landorus-therian
try:
  legend_list.remove('landorus')
  legend_list.extend(['landorus-incarnate', 'landorus-therian'])
except ValueError:
  print('Error')

# keldeo: keldeo-ordinary, keldeo-resolute
try:
  legend_list.remove('keldeo')
  legend_list.extend(['keldeo-ordinary', 'keldeo-resolute'])
except ValueError:
  print('Error')

# meloetta: meloetta-aria, meloetta-pirouette
try:
  legend_list.remove('meloetta')
  legend_list.extend(['meloetta-aria', 'meloetta-pirouette'])
except ValueError:
  print('Error')

# meltan: remove for now
try:
  legend_list.remove('meltan')
except ValueError:
  print('Error')

# melmetal: remove for now
try:
  legend_list.remove('melmetal')
except ValueError:
  print('Error')

## Test again for validity

In [ ]:
for p in legend_list:
  try:
    pb.pokemon(p)
  except ValueError:
    print("Cannot find", p)

## Obtain list of types

In [ ]:
# scrape list of legendary and mystical pokemon from bulbapedia
page = requests.get("https://bulbapedia.bulbagarden.net/wiki/Type")
soup = BeautifulSoup(page.content, 'html.parser')

# list of <a> tags of types
a_list = soup.select('td a[title*=(type)]')

# extract text from <a> tags
type_list = [lp.get_text() for lp in a_list]

# ignore ??? type
type_list = type_list[:-1]
type_list

In [ ]:
pstat = []
for a in legend_list:
  pstat.append()
  
  
  
  
rayquaza = pb.pokemon('rayquaza')
print(rayquaza.height)
print(rayquaza.weight)
  